In [24]:
import pandas as pd

df = pd.read_csv("Results.csv")

df

,Text_ID,Project,Free_Text_KEY,Free_Text_title,Text,Is_latest_text
0,3,IDN0174,5,Project Objective,To recover the social and economic activity in...,0
1,12843,TUN1004,5,Project Objective,the objective is to address the urgent needs o...,1
2,12868,JOR1001,5,Project Objective,The objective of this project is to promote re...,1
3,12870,BHR1002,5,Project Objective,The objective of the project is to promote you...,1
4,12871,YEM1001,5,Project Objective,The objective of the Program is to contribute ...,1
...,...,...,...,...,...,...
27208,185127,PAK1060,459,Readiness,The project is part of the Work Program for 20...,1
27209,185129,MOZ1022,5,Project Objective,To organize cataract screening campaigns and p...,1
27210,185130,MOZ1022,7,Scope,"To perform screening 10,000 patients and Condu...",1
27211,185131,MOZ1022,6,Project Locations,Mozabique,1


In [25]:
selected_fields = ['Scope' , 'Project Relevance' , 'Project Context' , 'Project Objective' , 'Project Rationale']

#selected_fields = [ 'Project Objective' ]

filtered_df = df[df['Free_Text_title'].isin(selected_fields) & df['Is_latest_text'] == 1 ]

df2 = filtered_df[['Project' , 'Free_Text_title' , 'Text']].pivot(index = ['Project'] , columns='Free_Text_title' , values = 'Text').dropna()

df2.reset_index(inplace=True)

In [26]:
#df3 = df2[['Project' , 'Project Objective']]

#df = 

df2.assign(Summary = 'ProjectNo:'+df2['Project'] + ' Objective:'+ df2['Project Objective'])

df2.reset_index(inplace=True)

#df3['Summary'] = 'ProjectNo:'+df3['Project'] + 'Objective:'+ df3['Project Objective']

In [21]:
#pd.unique(df2.Free_Text_title)

In [30]:
df_in = df2[['Project' , 'Project Objective']].rename(columns={'Project' : 'source'})


#df_in = df2[['Project' , 'Summary']].rename(columns={'Project' : 'source'})

df_in.head(5)
#df_in = 

Free_Text_title,source,Project Objective
0,AFG1025,The overall objective of this intervention is ...
1,AFG1026,The overall objective of this intervention is ...
2,AFG1027,The overall goal of this pilot intervention is...
3,AFG1030,Contribute to increased access to safe and sus...
4,AFG1031,"Provide essential Reproductive, Maternal, Neon..."


In [23]:
from langchain.document_loaders import DataFrameLoader


loader = DataFrameLoader(df_in, page_content_column="Project Objective")

docs = loader.load()

In [24]:
import os
import openai

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

#store = Chroma.from_documents(docs, embeddings)

store = Chroma.from_documents(docs, embeddings, collection_name='projects', persist_directory="./chroma_db")
store.persist()

In [26]:
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings, collection_name='projects' )

In [27]:
db.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'source': 'AFG1037'}),
 Document(page_content='The overall goal of this pilot intervention is to support vulnerable livelihoods and reduce food insecurity in vulnerable communities in southern Afghanistan.\nThe project is also expected to support the reduction of illicit crops cultivation, while, at the same time, promoting the economic empowerment of women.', metadata={'source': 'AFG1027'}),
 Document(page_content='Improve food security of vulnerable households through diversified Livelihood opportunities and Increase Agriculture and Livestock Production as main sources of livelihood for targeted communities.', metadata={'source': 'AFG1032'}),
 Document(page_content=' Project Development Objective is as f

In [8]:
store.similarity_search("Which projects eleviate poverty?")

[Document(page_content='Support vulnerable households in rural communities to have improved food security, household income, and resilience to climate change through the construction of agro-based infrastructure projects in the provinces of Badakhshan, Faryab, and Ghazni.', metadata={'source': 'AFG1037'}),
 Document(page_content='The overall goal of this pilot intervention is to support vulnerable livelihoods and reduce food insecurity in vulnerable communities in southern Afghanistan.\nThe project is also expected to support the reduction of illicit crops cultivation, while, at the same time, promoting the economic empowerment of women.', metadata={'source': 'AFG1027'}),
 Document(page_content='Improve food security of vulnerable households through diversified Livelihood opportunities and Increase Agriculture and Livestock Production as main sources of livelihood for targeted communities.', metadata={'source': 'AFG1032'}),
 Document(page_content=' Project Development Objective is as f

In [9]:
#from langchain.llms import OpenAIChat
from langchain.chat_models import ChatOpenAI

from langchain.chains import RetrievalQA

retriever = store.as_retriever()

#llm = OpenAIChat(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

llm = ChatOpenAI(temperature = 0.0, model_name="gpt-3.5-turbo-16k-0613")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [25]:
query = "Are there projects related to clean energy?"

res = qa_chain.run(query)

print(res)



> Entering new RetrievalQA chain...

> Finished chain.
Based on the given context, there is no specific mention of projects related to clean energy. The mentioned projects focus on enhancing access to clean water and improving food security, household income, and resilience to climate change through agro-based infrastructure projects.


In [26]:
from IPython.display import display, Markdown
display(Markdown(res.replace('.' ,'.\n\n')))


Based on the given context, there is no specific mention of projects related to clean energy.

 The mentioned projects focus on enhancing access to clean water and improving food security, household income, and resilience to climate change through agro-based infrastructure projects.



In [75]:
from langchain.chains import RetrievalQAWithSourcesChain

chain_ws = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

query = "Provide a short summary on the projects related to housing"

res = chain_ws("question: Provide a short summary on the projects related to housing")

print(res)


{'question': 'question: Provide a short summary on the projects related to housing', 'answer': 'There are several projects related to housing:\n\n1. Project in Bangladesh: The objective of this project is to expand access to affordable finance for constructing sustainable multi-storied housings with improved quality and necessary basic facilities for lower and middle-income people dwelling in rural and peri-urban areas. It also aims to address climate change by introducing climate-resilient and eco-friendly housing.\n\n2. Project in Uzbekistan: This project aims to improve access to quality education by improving the learning environment, including the construction and equipment of new schools. It also focuses on improving the quality and efficiency of education services and the quality of pre-school education.\n\n3. Project in Afghanistan: The goal of this project is to support vulnerable households in rural communities to have improved food security, household income, and resilience 

In [79]:

display(Markdown(res.replace('.' ,'.\n\n')))
display(res)


{'question': 'question: Provide a short summary on the projects related to housing',
 'answer': 'There are several projects related to housing:\n\n1. Project in Bangladesh: The objective of this project is to expand access to affordable finance for constructing sustainable multi-storied housings with improved quality and necessary basic facilities for lower and middle-income people dwelling in rural and peri-urban areas. It also aims to address climate change by introducing climate-resilient and eco-friendly housing.\n\n2. Project in Uzbekistan: This project aims to improve access to quality education by improving the learning environment, including the construction and equipment of new schools. It also focuses on improving the quality and efficiency of education services and the quality of pre-school education.\n\n3. Project in Afghanistan: The goal of this project is to support vulnerable households in rural communities to have improved food security, household income, and resilience

In [89]:
t = df_in[ df_in['source'] == 'BGD1087' ]

## Chain with Custom Prompt

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer clearly, using internation development related jargon."""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [ ]:
res = query = "Provide a short summary on the projects related to housing"
qa.run(query)

display(Markdown(res.replace('.' ,'.\n\n')))